In [26]:
import pandas as pd 
import numpy as np
import scipy.stats as stats

In [3]:
df_sales = pd.read_csv('2022-04-01T12_df_sales.csv', parse_dates = ['date'])
df_experiment = pd.read_csv('experiment_users.csv')

## Task 1

In [34]:
df_metrics = (
    df_sales
    .query("date >= '2022-03-16' & date < '2022-03-31'")
    .query("user_id in @df_experiment.query('pilot == 1').user_id")
    .assign(group = lambda x: np.where(x.date < '2022-03-24', 'before', 'after'))
    .groupby(['group', 'user_id'], as_index=False)
    .agg(revenue = ('price', 'sum'))
)

(
    df_metrics
    .groupby('group', as_index=False)
    .agg(
        avg_revenue = ('revenue', 'mean'),
        stg = ('revenue', 'std')
    )
)

,group,avg_revenue,stg
0,after,1251.512401,843.678374
1,before,1258.554642,852.817102


In [36]:
before = df_metrics.query("group == 'before'").revenue
after = df_metrics.query("group == 'after'").revenue

stats.ttest_ind(before, after, equal_var=False)

Ttest_indResult(statistic=0.3632346526454707, pvalue=0.7164474603058774)

## Task 2